## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,48.20,20,27.63,few clouds
1,1,Morondava,MG,-20.2833,44.2833,78.87,1,5.19,moderate rain
2,2,Vao,NC,-22.6667,167.4833,75.02,100,22.17,overcast clouds
3,3,Padina,RS,45.1181,20.7272,42.01,75,6.91,light rain
4,4,Araouane,ML,18.9048,-3.5265,90.86,0,0.65,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & 
                                       (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Cloudiness,Wind Speed,Current Description
1,1,Morondava,MG,-20.2833,44.2833,78.87,1,5.19,moderate rain
2,2,Vao,NC,-22.6667,167.4833,75.02,100,22.17,overcast clouds
8,8,Doctor Juan Manuel Frutos,PY,-25.3667,-55.8000,88.68,63,4.32,broken clouds
18,18,Hithadhoo,MV,-0.6000,73.0833,82.63,69,5.32,broken clouds
20,20,Salalah,OM,17.0151,54.0924,80.60,0,5.75,clear sky
21,21,Butaritari,KI,3.0707,172.7902,81.63,100,9.57,light rain
22,22,Rikitea,PF,-23.1203,-134.9692,77.77,9,7.36,clear sky
27,27,Kavieng,PG,-2.5744,150.7967,80.53,39,4.16,scattered clouds
35,35,Road Town,VG,18.4167,-64.6167,86.00,40,12.66,scattered clouds
45,45,Kapaa,US,22.0752,-159.3190,75.99,75,14.97,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                231
City                   231
Country                228
Lat                    231
Lng                    231
Max Temp               231
Cloudiness             231
Wind Speed             231
Current Description    231
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Morondava,MG,78.87,moderate rain,-20.2833,44.2833,
2,Vao,NC,75.02,overcast clouds,-22.6667,167.4833,
8,Doctor Juan Manuel Frutos,PY,88.68,broken clouds,-25.3667,-55.8000,
18,Hithadhoo,MV,82.63,broken clouds,-0.6000,73.0833,
20,Salalah,OM,80.60,clear sky,17.0151,54.0924,
21,Butaritari,KI,81.63,light rain,3.0707,172.7902,
22,Rikitea,PF,77.77,clear sky,-23.1203,-134.9692,
27,Kavieng,PG,80.53,scattered clouds,-2.5744,150.7967,
35,Road Town,VG,86.00,scattered clouds,18.4167,-64.6167,
45,Kapaa,US,75.99,broken clouds,22.0752,-159.3190,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")
        

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))